In [15]:
from DBEngine import db_engine
from DataService import get_raw, normalize
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset
from NeuralNetwork import NeuralNet, device

In [16]:
data = db_engine.execute(get_raw)

predict_df = pd.DataFrame(data, columns=["id", "lat", "lon", "date", "brightness", "brightness_t31", "radiative_power", "dist_road", "dist_rail", "avg_population"])
print("Available data: " + str(len(predict_df)))

Available data: 121555


In [17]:
predict_df = normalize(predict_df)
predict_np = predict_df.to_numpy()[:, 1:]

prediction_dataset = TensorDataset(
  torch.tensor(predict_np.astype('float64'), dtype=torch.float)
)

predict_df.head(10)

,id,lat,lon,date,brightness,brightness_t31,radiative_power,dist_road,dist_rail,avg_population
0,21969,0.835786,0.74169,0.802469,0.158597,0.486374,0.008338,0,0,0.000668
1,22253,0.866848,0.399656,0.803841,0.051502,0.377732,0.005154,0,0,0.000878
2,22721,0.797234,0.33169,0.817558,0.20034,0.481857,0.008049,0.000006,0,0.001126
3,22779,0.695104,0.291772,0.820302,0.14399,0.440251,0.006291,0.007102,0,0.001042
4,23187,0.859784,0.341707,0.823045,0.287666,0.505101,0.06074,0.905414,0,0.003798
5,23593,0.824422,0.628279,0.827160,0.189574,0.453002,0.026977,0.063796,0,0.004647
6,23890,0.921521,0.530906,0.832647,0.199836,0.451909,0.058848,0.020911,0.000272,0.001963
7,24319,0.855295,0.388451,0.846365,0.134169,0.441781,0.012391,0,0,0.002776
8,25181,0.098034,0.439774,0.582990,0.195177,0.320169,0.001977,0,0,0.008053
9,25656,0.511978,0.5442,0.596708,0.28817,0.313538,0.00723,0.695623,0,0.000271


In [18]:
model = NeuralNet().to(device)
model.load('./firms.pt')
print(model)

NeuralNet(
  (f_connected1): Linear(in_features=9, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=2, bias=True)
)


In [19]:
predicted = model.predict(prediction_dataset)
classes = predicted[0].detach().numpy()

In [20]:
predict_df['accident'] = classes[:, 0]
predict_df['attack'] = classes[:, 1]
predict_df['intent'] = np.where(predict_df['accident'] - predict_df['attack'] > 0, 1, 2)

,id,lat,lon,date,brightness,brightness_t31,radiative_power,dist_road,dist_rail,avg_population,accident,attack,intent
3365,24588,0.470287,0.76975,0.887517,0.043254,0.346328,0.016705,0.732836,0.000006,0.489226,-32.710724,32.785610,2
100011,133382,0.462465,0.768417,0.843621,0.217654,0.450306,0.00101,0.550077,0.293793,0.577596,-31.506607,31.589687,2
86718,119905,0.462489,0.763424,0.786008,0.21224,0.439959,0.002231,0.739842,0.71208,0.815698,-31.111996,31.209522,2
36657,63794,0.462489,0.763424,0.786008,0.21224,0.439959,0.002231,0.738296,0.713468,0.815722,-31.110104,31.207676,2
52284,79673,0.719839,0.437306,0.898491,0.453315,0.273827,0.008437,0.555189,0.018591,0.19395,-30.801525,30.863758,2
52877,80269,0.71996,0.436902,0.917695,0.246805,0.237904,0.004617,0.069177,0.075512,0.197647,-30.673500,30.745331,2
52955,80347,0.720272,0.436547,0.923182,0.274822,0.297144,0.001716,0.332674,0.400243,0.195808,-30.508556,30.583576,2
52361,79751,0.719735,0.437126,0.898491,0.453315,0.336855,0.004144,0.209134,0.005454,0.199013,-30.423428,30.488636,2
102661,136108,0.719943,0.436783,0.921811,0.21545,0.304284,0.003834,0.153252,0.061115,0.199906,-30.376511,30.446716,2
52391,79781,0.716272,0.52122,0.899863,0.311402,0.349825,0.002619,0.062884,0,0.223141,-29.998760,30.068802,2


In [21]:
predict_df.sort_values(by=['attack'], ascending=False).head(10)

,id,lat,lon,date,brightness,brightness_t31,radiative_power,dist_road,dist_rail,avg_population,accident,attack,intent
3365,24588,0.470287,0.76975,0.887517,0.043254,0.346328,0.016705,0.732836,0.000006,0.489226,-32.710724,32.785610,2
100011,133382,0.462465,0.768417,0.843621,0.217654,0.450306,0.00101,0.550077,0.293793,0.577596,-31.506607,31.589687,2
86718,119905,0.462489,0.763424,0.786008,0.21224,0.439959,0.002231,0.739842,0.71208,0.815698,-31.111996,31.209522,2
36657,63794,0.462489,0.763424,0.786008,0.21224,0.439959,0.002231,0.738296,0.713468,0.815722,-31.110104,31.207676,2
52284,79673,0.719839,0.437306,0.898491,0.453315,0.273827,0.008437,0.555189,0.018591,0.19395,-30.801525,30.863758,2
52877,80269,0.71996,0.436902,0.917695,0.246805,0.237904,0.004617,0.069177,0.075512,0.197647,-30.673500,30.745331,2
52955,80347,0.720272,0.436547,0.923182,0.274822,0.297144,0.001716,0.332674,0.400243,0.195808,-30.508556,30.583576,2
52361,79751,0.719735,0.437126,0.898491,0.453315,0.336855,0.004144,0.209134,0.005454,0.199013,-30.423428,30.488636,2
102661,136108,0.719943,0.436783,0.921811,0.21545,0.304284,0.003834,0.153252,0.061115,0.199906,-30.376511,30.446716,2
52391,79781,0.716272,0.52122,0.899863,0.311402,0.349825,0.002619,0.062884,0,0.223141,-29.998760,30.068802,2


In [22]:
predict_df.sort_values(by=['accident'], ascending=False).head(10)

,id,lat,lon,date,brightness,brightness_t31,radiative_power,dist_road,dist_rail,avg_population,accident,attack,intent
102774,23,0.395735,0.136478,0.076818,0.052257,0.382614,0.006072,0.149089,0,0.008127,39.575737,-39.508835,1
102790,90,0.4051,0.136428,0.091907,0.032739,0.344725,0.003813,0.978666,0,0.006994,39.215786,-39.138783,1
102789,88,0.399054,0.134926,0.091907,0.07996,0.333795,0.012497,0.532552,0,0.006628,38.684521,-38.614067,1
102793,93,0.399834,0.13798,0.091907,0.046591,0.341081,0.00473,0.246656,0,0.006601,38.679451,-38.613232,1
102792,92,0.400603,0.138193,0.091907,0.051628,0.335252,0.005154,0.021612,0,0.006252,38.521080,-38.458462,1
102791,91,0.400205,0.137492,0.091907,0.081849,0.34181,0.006354,0.022218,0,0.006105,38.398922,-38.335957,1
102780,45,0.556403,0.228766,0.083676,0.04848,0.346911,0.012567,0,0,0.001497,38.057648,-37.990356,1
102776,34,0.708863,0.123835,0.078189,0.121514,0.299548,0.008331,0,0,0.003929,38.017311,-37.948467,1
102779,44,0.556796,0.228791,0.083676,0.064849,0.330152,0.009108,0,0,0.001522,37.833752,-37.766930,1
102816,392,0.402091,0.136791,0.115226,0.049109,0.35274,0.027323,0.694616,0,0.007007,37.478027,-37.404755,1


In [23]:
def update(cursor):
  query = "update geo_firms set source=6, accident=%(accident)s, attack=%(attack)s, intent=%(intent)s where id = %(id)s"
  cursor.executemany(query, predict_df.to_dict('records'))

db_engine.execute(update)
db_engine.commit()